In [1]:
# import general classes for data processing
import ruptures as rpt
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import seaborn as sns

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# import our custom classes
import sys
path_to_directory = '../'
if path_to_directory not in sys.path:
    sys.path.append(path_to_directory)
from model import Signal, GestureRecognitionModel
from model import GestureRecognitionDataset

# import MLP
from sklearn.neural_network import MLPClassifier

# import for loop and XGboost
from tqdm.notebook import tqdm
from xgboost import XGBClassifier

# import model saving
import joblib

In [2]:
df = pd.read_csv('assets\data_rem.csv')

subjects = list(df.subject.unique())
subjects.remove("Mario")
subjects.remove("Hanaa")
subjects.remove("Alex")
subjects.remove("David")
subjects.remove("Francisco")

df = df[df['subject'].isin(subjects)]
subjects

['Aarushi', 'Ester', 'Imen', 'Lluis', 'Sergi', 'Sergio']

In [3]:
test_sets = [1, 2, 3]
train_sets = [1, 2, 3]

subjects = list(df.subject.unique())

accuracy_matrix = np.zeros((len(subjects), 1, len(train_sets), len(test_sets), 2))
# Dimension breakdown:
# len(subjects) - Number of subjects
# 1 - Single value for accuracy_pre per subject
# len(train_sets) - Number of training sets
# len(test_sets) - Number of testing sets
# 2 - Accuracy_pre and Accuracy_post

for subject_idx, person_test in  tqdm(enumerate(subjects)):

    model = MLPClassifier(random_state=1,hidden_layer_sizes =50,max_iter=500)

    selected_columns = [ 'mav_0', 'mav_1', 'mav_2', 'mav_3', 'mav_4', 'mav_5', 'mav_6', 'mav_7',
    'rms_0', 'rms_1', 'rms_2', 'rms_3', 'rms_4', 'rms_5', 'rms_6', 'rms_7',
    'ssc_0', 'ssc_1', 'ssc_2', 'ssc_3', 'ssc_4', 'ssc_5', 'ssc_6', 'ssc_7',
    'wl_0', 'wl_1', 'wl_2', 'wl_3', 'wl_4', 'wl_5', 'wl_6', 'wl_7', 'var_0',
    'var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'var_6', 'var_7', 'iasd_0',
    'iasd_1', 'iasd_2', 'iasd_3', 'iasd_4', 'iasd_5', 'iasd_6', 'iasd_7',
    'iatd_0', 'iatd_1', 'iatd_2', 'iatd_3', 'iatd_4', 'iatd_5', 'iatd_6',
    'iatd_7']

    df_train = df[(df['subject'] != person_test)]
    df_test = df[(df['subject'] == person_test)]

    X_train = df_train[selected_columns]
    y_train = df_train['gesture']
    X_test = df_test[selected_columns]
    y_test = df_test['gesture']        

    label_encoder = LabelEncoder()
    Y_encoded = label_encoder.fit_transform(y_train)
    y_train = Y_encoded 

    Y_encoded2 = label_encoder.fit_transform(y_test)
    y_test = Y_encoded2 

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    predictions = [round(value) for value in y_pred]
    predicted_labels_decoded = label_encoder.inverse_transform(predictions)

    accuracy_pre = accuracy_score(y_test, predictions)*100
    accuracy_matrix[subject_idx, 0, 0, 0, 0] = accuracy_pre

    for train_idx, Take_to_train in enumerate(train_sets):
        for test_idx, Take_to_test in enumerate(test_sets):

            df2 = df[(df['subject'] == person_test)] 
            model2 = model

            df_train2 = df2[(df2['take'] == Take_to_train)]
            df_test2 = df2[(df2['take'] == Take_to_test)]

            X_train2 = df_train2[selected_columns]
            y_train2 = df_train2['gesture']
            X_test2 = df_test2[selected_columns]
            y_test2 = df_test2['gesture']

            label_encoder = LabelEncoder()
            Y_encoded = label_encoder.fit_transform(y_train2)
            y_train2 = Y_encoded 

            Y_encoded2 = label_encoder.fit_transform(y_test2)
            y_test2 = Y_encoded2 
            

            model2.partial_fit(X_train2, y_train2, classes=np.unique(y_train2))

            y_pred = model2.predict(X_test2)

            predictions = [round(value) for value in y_pred]
            predicted_labels_decoded = label_encoder.inverse_transform(predictions)
            
            accuracy_post = accuracy_score(y_test2, predictions)*100

            accuracy_matrix[subject_idx, 0, train_idx, test_idx, 1] = accuracy_post

0it [00:00, ?it/s]

c:\Users\fents\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [11]:
accuracy_matrix = np.load('assets\accuracy_matrix.npy')

results_matrix = np.zeros((6, 2))

# selecting relevant accuracies in constructed matrices
for i in range(6):

    sub = accuracy_matrix[i:i+1]
    Pretrained_acc = sub[0][0][0][0][0]
   
    # remove the specified numbers
    sub[0, 0, 0, 0, 1] = 0  # remove the first number of the first subsection
    sub[0, 0, 1, 1, 1] = 0  # remove the second number of the second subsection
    sub[0, 0, 2, 2, 1] = 0  # remove the third number of the third subsection

    second_column = sub[0, 0, :, :, 1]
    non_zero_values = second_column[second_column != 0]
    average_non_zero = np.mean(non_zero_values)

    results_matrix[i, 0] = Pretrained_acc
    results_matrix[i, 1] = average_non_zero

labeled_data = np.vstack((["pretrained", "finetuned"], results_matrix))
print(labeled_data)
np.save('Average_accuracies',results_matrix)

[['pretrained' 'finetuned']
 ['42.36695988520806' '71.22300567186211']
 ['21.569201493083508' '55.520216280398586']
 ['47.09792709077913' '63.79681947931322']
 ['56.59416299559471' '75.54055427607591']
 ['37.91985007928499' '51.90336606473116']
 ['44.07455091655938' '56.112641421627444']]


In [13]:
# same process for xgboost:

df = pd.read_csv('assets\data_rem.csv')
subjects = list(df.subject.unique())
subjects.remove("Mario")
subjects.remove("Hanaa")
subjects.remove("Alex")
subjects.remove("David")
subjects.remove("Francisco")

df = df[df['subject'].isin(subjects)]

selected_columns = [ 'mav_0', 'mav_1', 'mav_2', 'mav_3', 'mav_4', 'mav_5', 'mav_6', 'mav_7',
       'rms_0', 'rms_1', 'rms_2', 'rms_3', 'rms_4', 'rms_5', 'rms_6', 'rms_7',
       'ssc_0', 'ssc_1', 'ssc_2', 'ssc_3', 'ssc_4', 'ssc_5', 'ssc_6', 'ssc_7',
       'wl_0', 'wl_1', 'wl_2', 'wl_3', 'wl_4', 'wl_5', 'wl_6', 'wl_7', 'var_0',
       'var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'var_6', 'var_7', 'iasd_0',
       'iasd_1', 'iasd_2', 'iasd_3', 'iasd_4', 'iasd_5', 'iasd_6', 'iasd_7',
       'iatd_0', 'iatd_1', 'iatd_2', 'iatd_3', 'iatd_4', 'iatd_5', 'iatd_6',
       'iatd_7']
            

train_sets = [1, 2, 3]

subjects = list(df.subject.unique())
xgboost_accuracies = [[0 for _ in range(len(train_sets))] for _ in range(len(subjects))]

for subject_idx, person_test in  tqdm(enumerate(subjects)):
    for train_idx, Take_to_test in enumerate(train_sets):
          
        df2 = df[(df['subject'] == person_test)]
        model = XGBClassifier()

        df_train = df2[(df2['take'] != Take_to_test)]
        df_test = df2[(df2['take'] == Take_to_test)]
     
        X_train = df_train[selected_columns]
        y_train = df_train['gesture']
        X_test = df_test[selected_columns]
        y_test = df_test['gesture']

        label_encoder = LabelEncoder()
        Y_encoded = label_encoder.fit_transform(y_train)
        y_train = Y_encoded 

        Y_encoded2 = label_encoder.fit_transform(y_test)
        y_test = Y_encoded2 

        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)

        predictions = [round(value) for value in y_pred]
        predicted_labels_decoded = label_encoder.inverse_transform(predictions)

        accuracy = accuracy_score(y_test, predictions)*100
        xgboost_accuracies[subject_idx][train_idx] = accuracy

row_averages = np.mean(xgboost_accuracies, axis=1)

print("Average of each subject:", row_averages)

0it [00:00, ?it/s]

Average of each subject: [75.87890459 57.51607498 69.00701482 79.67279265 56.9820413  66.35050804]


In [ ]:
np.save('Average_accuracies_xgboost',xgboost_accuracies)